# Binance trading bot - Prototyping

In [3]:
# Prototype built in this note book

# -->  bot.py is the actual tradeing bot
# -->  dashboard.py to visualize ticker and trades

# Thanks to parttimelarry (hackingthemarkets.com) for inspiration and awesome tutorials

In [4]:
import os, csv, datetime, websocket, json, pprint
from binance.client import Client 
from binance.enums import *
import numpy as np
import pandas as pd
import pandas_ta as pta
#import talib
#import backtrader as bt
import config
import pickle
import datetime
from tradingview_ta import TA_Handler, Interval

In [5]:
# Initialise the API client
client = Client(config.API_KEY, config.API_SECRET)

## Strategy

In [10]:
### STRATEGY FUNCTIONS ###

# To get recommendation from Tradingview api
def tradingview_rec(coin_pair, period):
    coin = TA_Handler()
    coin.set_symbol_as(coin_pair)
    coin.set_exchange_as_crypto_or_stock("BINANCE")
    coin.set_screener_as_crypto()
    
    if period == "hour":
        coin.set_interval_as(Interval.INTERVAL_1_HOUR)
    if period == "day":
        coin.set_interval_as(Interval.INTERVAL_1_DAY)
    elif period == "week":
        coin.set_interval_as(Interval.INTERVAL_1_WEEK)
    rec = coin.get_analysis().summary
    rec.update({"COIN_PAIR": coin_pair})
    rec.update({"PERIOD": period})
    
    return rec



def my_strategy(df, coin_pair, period):
    
    ### SETTINGS ###
    MA_PERIOD      = 50
    MA_OFFSET_SELL = 1
    MA_OFFSET_BUY  = 1
    RSI_PERIOD     = 14
    RSI_OVERBOUGHT = 75
    RSI_OVERSOLD   = 25
    
    df["c"] = df["c"].astype(float)
    # Current price
    last_close = df["c"].iloc[-1]
    print(f"strategy looks at: {last_close}")
    
    # MA
    df["ma"] = df["c"].rolling(window=MA_PERIOD).mean()
    last_ma = df["ma"].iloc[-1]
    
    # RSI
    rsi = pta.rsi(df["c"], length=RSI_PERIOD)
    last_rsi = rsi.iloc[-1].round(1)
    
    # Get TradingView recommendation
    tradingview_say = tradingview_rec(coin_pair, period)
    if tradingview_say["RECOMMENDATION"] == "BUY" or tradingview_say["RECOMMENDATION"] == "STRONG_BUY" :
        tradingview_buy = True 
    elif tradingview_say["RECOMMENDATION"] == "SELL" or tradingview_say["RECOMMENDATION"] == "STRONG_SELL" :
        tradingview_buy = False
        
    # Conditions for 'sell' or 'buy' or 'wait'
    if (last_rsi > RSI_OVERBOUGHT or last_close < (last_ma * MA_OFFSET_SELL)) and tradingview_buy == False:
        print(f"the current rsi_{RSI_PERIOD} is: ", last_rsi)
        print(f"the current ma_{MA_PERIOD} is: ", last_ma)
        print("tradingview say: ", tradingview_say["RECOMMENDATION"])
        
        return "sell"
    
    
    elif (last_rsi < RSI_OVERSOLD or last_close > (last_ma * MA_OFFSET_BUY)) and tradingview_buy == True:
        print(f"the current rsi_{RSI_PERIOD} is", last_rsi)
        print(f"the current ma_{MA_PERIOD} is", last_ma)
        print("tradingview say: ", tradingview_say["RECOMMENDATION"])
        
        return "buy"
    
    
    else:
        print(f"the current rsi_{RSI_PERIOD} is", last_rsi)
        print(f"the current ma_{MA_PERIOD} is", last_ma)
        print("tradingview say: ", tradingview_say["RECOMMENDATION"])
        
        return "wait"
    
    

In [11]:
# To place order for Binance api
def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True


## Run the bot

In [12]:
### RUNNNG THE BOT ###

### SETTINGS ###
MIN_START_TIME_PERODS = 14       # Start trading after this num of periods (candles)
TARGET_PERIOD = "hour"           # Used bt tradingWiew api to get recommendations (hour/day/week)
TRADE_SYMBOL = "ETHUSDT"
TIME_RESOLUTION = "5m"           # note its related to TA-indicators (MA/RSI)
PLACE_LIVE_ORDER = False         # FALSE mean no order will be placed (but still visualized)
TRADE_QUANTITY = 0.5             # how much to trade (crypto coins)

##########
START_CAPITAL = 10_000               # USDT
###########


SOCKET = f"wss://stream.binance.com:9443/ws/{TRADE_SYMBOL.lower()}@kline_{TIME_RESOLUTION}"

# Initiate list of candle sequences, sells, buys and balance
candles = []

# init lists - to check and visualize performance and earnings
sells = []
buys = []

balances_usdt_lst = []
balances_coin_lst = []
in_position = False # at start, modified after order

balance_usdt = START_CAPITAL  # modified depending on trades
balance_coin = 0  # modified depending on trades

client = Client(config.API_KEY, config.API_SECRET)

def on_open(ws):
    print('open connection')

def on_close(ws):
    print('close connection')

def on_message(ws, message):
    global closes, in_position, balance_usdt, balance_coin # since function is called on message
    
    #print('received message')
    json_message = json.loads(message)
    candle = json_message['k']

    candle_closed = candle['x']
    close = candle['c']

    if candle_closed:
        print("--------------")
        print(f"Candle closed at {close}")
        print(f"In position: {in_position}")
        
        print(f"balance_usdt: {balance_usdt}")
        print(f"balance_coin: {balance_coin}, value: {balance_coin * float(close)} usd")
            
        
        candles.append(candle)
        df = pd.DataFrame(candles)
        
        print("shape of df", df.shape)
        #print("df mem: ", df.memory_usage(deep=True).sum())
        print("UTC Time: ", datetime.datetime.utcnow())
        

        if len(candles) > MIN_START_TIME_PERODS:
            
            try:
                strategy = my_strategy(df, TRADE_SYMBOL, TARGET_PERIOD)
                print("Strategy say: ", strategy)
            except Exception as e:
                print("hmm, an exception occured - {}".format(e))
   
            if strategy == "sell":
                if in_position:
                    #binance sell logic
                    if PLACE_LIVE_ORDER == True:
                        print("I AM SELLING 'LIVE' !")
                        order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = False
                            
                    elif PLACE_LIVE_ORDER == False:
                        print("I AM SELLING 'TEST' !")
                        in_position = False
                    
                    # append time and price to buy list
                    sells.append({df["T"].iloc[-1]: df["c"].iloc[-1]})
                    
                    # append to balances lists
                    balance_usdt = balance_usdt + (float(close) * TRADE_QUANTITY)
                    balances_usdt_lst.append({df["T"].iloc[-1]: balance_usdt})
                    print("sold, - balance usdt (added to lst)", balance_usdt)
                    balance_coin =  balance_coin - TRADE_QUANTITY
                    balances_coin_lst.append({df["T"].iloc[-1]: balance_coin * float(close)})
                    print("sold, - balance coin (added to lst)", balance_coin)
                    
                else:
                    print("Sell Signal. But Not In Position ")
            
            
            if strategy == "buy":
                if in_position:
                    print("Buy Signal. But Already In Position")
                    
                else:
                    #binance buy order logic
                    if PLACE_LIVE_ORDER == True:
                        print("I AM BUYING 'LIVE' !")
                        order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = True 
                            
                    elif PLACE_LIVE_ORDER == False:
                        print("I AM BUYING 'TEST' !")
                        in_position = True
                        
                    # append time and price to buy list
                    buys.append({df["T"].iloc[-1]: df["c"].iloc[-1]})
                    
                    # append to balances lists
                    balance_usdt = balance_usdt - (float(close) * TRADE_QUANTITY)
                    balances_usdt_lst.append({df["T"].iloc[-1]: balance_usdt})
                    print("bought, - balance usdt (added to lst)", balance_usdt)
                    balance_coin =  balance_coin + TRADE_QUANTITY
                    balances_coin_lst.append({df["T"].iloc[-1]: balance_coin * float(close)})
                    print("bought, - balance coin (added to lst)", balance_coin)
                        

                        
          
            # Save df to be visualized in real time web app
            print("saving df..")
            df.to_csv(f"data/{SOCKET[41:]}_{TRADE_SYMBOL}.csv")
            print("df saved")
            pickle.dump(sells, open("data/sells.p", "wb"))
            pickle.dump(buys, open("data/buys.p", "wb"))
            pickle.dump(balances_usdt_lst, open("data/balances_usdt_lst.p", "wb"))
            pickle.dump(balances_coin_lst, open("data/balances_coin_lst.p", "wb"))
            pickle.dump([balance_usdt, balance_coin, START_CAPITAL], open("data/current_balance.p", "wb"))
            print("all buy/sell/balances lists saved")

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever() 

open connection
--------------
Candle closed at 3234.54000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (1, 17)
UTC Time:  2022-01-12 08:50:00.282078
--------------
Candle closed at 3236.70000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (2, 17)
UTC Time:  2022-01-12 08:54:59.390255
--------------
Candle closed at 3232.00000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (3, 17)
UTC Time:  2022-01-12 08:59:59.395322
--------------
Candle closed at 3247.99000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (4, 17)
UTC Time:  2022-01-12 09:04:59.390945
--------------
Candle closed at 3246.61000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (5, 17)
UTC Time:  2022-01-12 09:09:59.382170
--------------
Candle closed at 3242.52000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 

the current rsi_14 is 51.8
the current ma_50 is nan
tradingview say:  BUY
Strategy say:  wait
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3256.37000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (32, 17)
UTC Time:  2022-01-12 11:24:59.379753
strategy looks at: 3256.37
the current rsi_14 is 54.2
the current ma_50 is nan
tradingview say:  BUY
Strategy say:  wait
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3250.95000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (33, 17)
UTC Time:  2022-01-12 11:29:59.379178
strategy looks at: 3250.95
the current rsi_14 is 49.3
the current ma_50 is nan
tradingview say:  BUY
Strategy say:  wait
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3251.60000000
In position: False
balance_usdt: 10000
balance_coin: 0, value: 0.0 usd
shape of df (34, 17)
UTC Time:

the current rsi_14 is 60.0
the current ma_50 is 3270.1756
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3304.31000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1652.155 usd
shape of df (55, 17)
UTC Time:  2022-01-12 13:19:59.441363
strategy looks at: 3304.31
the current rsi_14 is 59.1
the current ma_50 is 3271.3296
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3306.95000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1653.475 usd
shape of df (56, 17)
UTC Time:  2022-01-12 13:24:59.383055
strategy looks at: 3306.95
the current rsi_14 is 60.4
the current ma_50 is 3272.6182
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
------

--------------
Candle closed at 3375.84000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1687.92 usd
shape of df (75, 17)
UTC Time:  2022-01-12 14:59:59.382879
strategy looks at: 3375.84
the current rsi_14 is 67.8
the current ma_50 is 3312.3572000000004
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3372.81000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1686.405 usd
shape of df (76, 17)
UTC Time:  2022-01-12 15:04:59.386290
strategy looks at: 3372.81
the current rsi_14 is 65.4
the current ma_50 is 3314.7660000000005
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3379.00000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1689.5 usd
shape of df (77, 17)
UTC Time:  2022-01-12 15:09:59.373

the current rsi_14 is 56.0
the current ma_50 is 3350.533
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3364.59000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1682.295 usd
shape of df (96, 17)
UTC Time:  2022-01-12 16:44:59.376093
strategy looks at: 3364.59
the current rsi_14 is 55.2
the current ma_50 is 3351.5378
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3366.99000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1683.495 usd
shape of df (97, 17)
UTC Time:  2022-01-12 16:49:59.370337
strategy looks at: 3366.99
the current rsi_14 is 57.0
the current ma_50 is 3352.603
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------

...

--------------
Candle closed at 3367.89000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1683.945 usd
shape of df (310, 17)
UTC Time:  2022-01-13 10:34:59.345437
strategy looks at: 3367.89
the current rsi_14 is 57.7
the current ma_50 is 3350.8556
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3365.21000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1682.605 usd
shape of df (311, 17)
UTC Time:  2022-01-13 10:39:59.339513
strategy looks at: 3365.21
the current rsi_14 is 54.8
the current ma_50 is 3351.3361999999997
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3362.65000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1681.325 usd
shape of df (312, 17)
UTC Time:  2022-01-13 10:44:59.337650

--------------
Candle closed at 3351.44000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1675.72 usd
shape of df (333, 17)
UTC Time:  2022-01-13 12:29:59.329068
strategy looks at: 3351.44
the current rsi_14 is 53.0
the current ma_50 is 3353.6711999999998
tradingview say:  BUY
Strategy say:  wait
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3354.80000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1677.4 usd
shape of df (334, 17)
UTC Time:  2022-01-13 12:34:59.339582
strategy looks at: 3354.8
the current rsi_14 is 55.9
the current ma_50 is 3353.86
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3357.69000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1678.845 usd
shape of df (335, 17)
UTC Time:  2022-01-13 12:39:59.330080
strategy looks at: 3357.69
the current r

the current rsi_14 is 61.3
the current ma_50 is 3356.877
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3375.03000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1687.515 usd
shape of df (354, 17)
UTC Time:  2022-01-13 14:14:59.328808
strategy looks at: 3375.03
the current rsi_14 is 61.6
the current ma_50 is 3357.065
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3384.72000000
In position: True
balance_usdt: 8347.675
balance_coin: 0.5, value: 1692.36 usd
shape of df (355, 17)
UTC Time:  2022-01-13 14:19:59.336847
strategy looks at: 3384.72
the current rsi_14 is 67.4
the current ma_50 is 3357.3712
tradingview say:  BUY
Strategy say:  buy
Buy Signal. But Already In Position
saving df..
df saved
all buy/sell/balances lists saved
-------

the current rsi_14 is:  28.1
the current ma_50 is:  3355.6834000000003
tradingview say:  SELL
Strategy say:  sell
Sell Signal. But Not In Position 
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3280.66000000
In position: False
balance_usdt: 10006.009999999998
balance_coin: 0.0, value: 0.0 usd
shape of df (374, 17)
UTC Time:  2022-01-13 15:54:59.425157
strategy looks at: 3280.66
the current rsi_14 is:  25.6
the current ma_50 is:  3354.4692000000005
tradingview say:  SELL
Strategy say:  sell
Sell Signal. But Not In Position 
saving df..
df saved
all buy/sell/balances lists saved
--------------
Candle closed at 3301.43000000
In position: False
balance_usdt: 10006.009999999998
balance_coin: 0.0, value: 0.0 usd
shape of df (375, 17)
UTC Time:  2022-01-13 15:59:59.369852
strategy looks at: 3301.43
the current rsi_14 is:  36.5
the current ma_50 is:  3353.7778000000003
tradingview say:  SELL
Strategy say:  sell
Sell Signal. But Not In Position 
saving d

False

# API Tradingview recommendations - Screener

In [8]:
from tradingview_ta import TA_Handler, Interval

def tradingview_rec(coin_pair, period):
    coin = TA_Handler()
    coin.set_symbol_as(coin_pair)
    coin.set_exchange_as_crypto_or_stock("BINANCE")
    coin.set_screener_as_crypto()
    if period == "hour":
        coin.set_interval_as(Interval.INTERVAL_1_HOUR)
    if period == "day":
        coin.set_interval_as(Interval.INTERVAL_1_DAY)
    elif period == "week":
        coin.set_interval_as(Interval.INTERVAL_1_WEEK)
    rec = coin.get_analysis().summary
    rec.update({"COIN_PAIR": coin_pair})
    rec.update({"PERIOD": period})
    return rec

def tradingview_screener(coin_list):
    screener = []
    for coin in coin_list:
        screener.append(tradingview_rec(coin, "hour"))
        screener.append(tradingview_rec(coin, "day"))
        screener.append(tradingview_rec(coin, "week"))
    return pd.DataFrame(screener)
    

In [9]:
coin_list = ["BTCUSDT", "ETHUSDT", "DOTUSDT", "DOGEUSDT", "SOLUSDT"]
tradingview_screener(coin_list)

,RECOMMENDATION,BUY,SELL,NEUTRAL,COIN_PAIR,PERIOD
0,NEUTRAL,10,7,9,BTCUSDT,hour
1,SELL,2,14,10,BTCUSDT,day
2,SELL,7,12,7,BTCUSDT,week
3,BUY,13,4,9,ETHUSDT,hour
4,SELL,1,15,10,ETHUSDT,day
5,NEUTRAL,9,10,7,ETHUSDT,week
6,BUY,14,2,10,DOTUSDT,hour
7,SELL,2,14,10,DOTUSDT,day
8,SELL,2,11,9,DOTUSDT,week
9,BUY,11,6,9,DOGEUSDT,hour


# Create Readme File

#  Crypto trade bot

## 1. What it is

Trading bot connected to Binance API.
#### This project made for fun. So ...  Do not use to trade live before you have backtested and modified/ created your own strategy function you are happy with.


## 2. How to

1. Update settings and strategy function in bot.py. By default it will not place live orders. ('place_live_orders = False')
2. Add your API key /secret in config.py
3. Run trade bot, bot.py
4. Run dashboard, dashboard.py. To view dashboard, open your browser and type localhost:8050

Prototyped in jupyter notebook where yoy can play with settings and strategy.

CLI:

![cli](pics/bot_cli_1.png)



## 3. What the Dashboard looks like

- Graph where you can see trades taken
- Follow how your cash/coin balance changes over time. Track trades based on startegy even though no live orders are placed
- Account balance from Binance



![graph](pics/trade_bot_dashboard.png)

-----------------------------------------
-----------------------------------------
-----------------------------------------


![graph2](pics/trade_bot_dashboard_2.png)
